In [32]:
# symbol.txtファイルを読み込んで、_SPECIALSに記号を追加
# https://github.com/reazon-research/ReazonSpeech/blob/master/pkg/_v1/src/data/symbol.txt
with open('symbol.txt', 'r') as f:
    _SPECIALS = {ord(c.rstrip("\n")): "" for c in f.readlines()}

_HAN2ZEN = str.maketrans(
    "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789",
    "ａｂｃｄｅｆｇｈｉｊｋｌｍｎｏｐｑｒｓｔｕｖｗｘｙｚＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺ０１２３４５６７８９")

def normalize(text):
    """Trim non-phonatory symbols in the text

    Args:
        text(str): A string to process

    Returns:
        A normalized string
    """
    text = text.replace('<sos/eos>', '')
    text = text.replace('<unk>', '')
    return text.translate(_SPECIALS).translate(_HAN2ZEN)


# cerを計測する関数を定義
import editdistance

def calculate_cer(reference_texts, predicted_texts):
    total_errors = 0
    total_chars = 0
    
    for ref, pred in zip(reference_texts, predicted_texts):
        # Levenshtein距離を使ってエディット距離を計算する
        levenshtein_distance = editdistance.eval(ref, normalize(pred))
        total_errors += levenshtein_distance
        total_chars += len(ref)
    
    # CERを計算する
    cer = total_errors / total_chars
    return cer

[' ', '!', '"', "'", '(', ')', ',', '-', '.', '/', ':', '?', '[', '\\', ']', '{', '‐', '–', '—', '―', '‘', '’', '“', '”', '…', '‼', '⁉', '−', '≪', '①', '②', '③', '─', '┐', '▲', '△', '◆', '○', '●', '★', '☆', '☎', '♥', '♪', '\u3000', '、', '。', '〈', '〉', '《', '》', '「', '」', '『', '』', '〜', '・', '！', '＃', '（', '）', '＊', '，', '－', '．', '／', '：', '？', '［', '］', '～', '｢', '｣', '･']


In [31]:
with open('symbol.txt', 'r') as f:
    print([line.rstrip("\n") for line in f.readlines()])
    _SPECIALS = {ord(c.rstrip("\n")): "" for c in f.readlines()}

[' ', '!', '"', "'", '(', ')', ',', '-', '.', '/', ':', '?', '[', '\\', ']', '{', '‐', '–', '—', '―', '‘', '’', '“', '”', '…', '‼', '⁉', '−', '≪', '①', '②', '③', '─', '┐', '▲', '△', '◆', '○', '●', '★', '☆', '☎', '♥', '♪', '\u3000', '、', '。', '〈', '〉', '《', '》', '「', '」', '『', '』', '〜', '・', '！', '＃', '（', '）', '＊', '，', '－', '．', '／', '：', '？', '［', '］', '～', '｢', '｣', '･']


In [30]:
print(_SPECIALS)

{32: '', 33: '', 34: '', 39: '', 40: '', 41: '', 44: '', 45: '', 46: '', 47: '', 58: '', 63: '', 91: '', 92: '', 93: '', 123: '', 8208: '', 8211: '', 8212: '', 8213: '', 8216: '', 8217: '', 8220: '', 8221: '', 8230: '', 8252: '', 8265: '', 8722: '', 8810: '', 9312: '', 9313: '', 9314: '', 9472: '', 9488: '', 9650: '', 9651: '', 9670: '', 9675: '', 9679: '', 9733: '', 9734: '', 9742: '', 9829: '', 9834: '', 12288: '', 12289: '', 12290: '', 12296: '', 12297: '', 12298: '', 12299: '', 12300: '', 12301: '', 12302: '', 12303: '', 12316: '', 12539: '', 65281: '', 65283: '', 65288: '', 65289: '', 65290: '', 65292: '', 65293: '', 65294: '', 65295: '', 65306: '', 65311: '', 65339: '', 65341: '', 65374: '', 65378: '', 65379: '', 65381: ''}


In [ ]:
# データのダウンロード

import pandas as pd
from pathlib import Path

data_dir = Path('/root/datadrive/TEDxJP-10K_v1.1')
wav_dir = data_dir / 'test_dump'
id2text = pd.read_csv(data_dir / 'text', sep=' ', header=None, names=['id', 'text']).set_index('id')['text'].to_dict()

In [20]:
from nemo.collections.asr.models import EncDecRNNTBPEModel
model = EncDecRNNTBPEModel.restore_from('/root/datadrive/reazonspeech-nemo-v2/reazonspeech-nemo-v2.nemo')

[NeMo I 2024-05-29 14:34:16 mixins:172] Tokenizer SentencePieceTokenizer initialized with 3000 tokens


[NeMo W 2024-05-29 14:34:17 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: dataset/train.json
    sample_rate: 16000
    batch_size: 32
    shuffle: true
    num_workers: 8
    pin_memory: true
    max_duration: 30
    min_duration: 0.1
    use_start_end_token: false
    trim_silence: false
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: fully_randomized
    bucketing_batch_size: null
    
[NeMo W 2024-05-29 14:34:17 modelPT:172] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: dataset/valid.json
    sample_rate: 16000
    batch_size: 16
    shuffle: false

[NeMo I 2024-05-29 14:34:17 features:289] PADDING: 0
[NeMo I 2024-05-29 14:34:20 rnnt_models:217] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2024-05-29 14:34:21 save_restore_connector:249] Model EncDecRNNTBPEModel was successfully restored from /root/datadrive/reazonspeech-nemo-v2/reazonspeech-nemo-v2.nemo.


In [10]:
import pandas as pd
import librosa
import soundfile as sf
import numpy as np
from pathlib import Path

data_dir = Path('/root/datadrive/TEDxJP-10K_v1.1')
wav_dir = data_dir / 'test_dump'
id2text = pd.read_csv(data_dir / 'text', sep=' ', header=None, names=['id', 'text']).set_index('id')['text'].to_dict()
id2text

{'-6K2nN9aWsg-00002686-00002940': '今大学教員をやってるんですけど',
 '-6K2nN9aWsg-00004949-00005608': 'それについて今日あのまー紹介したいとあー思ってます',
 '-6K2nN9aWsg-00007367-00007519': '周りをみると',
 '-6K2nN9aWsg-00010059-00010323': 'そういうこう読み書きできない人になってしまった',
 '-6K2nN9aWsg-00012362-00012656': 'そのフィルタが外されたような感じですね',
 '-6K2nN9aWsg-00014746-00014959': 'この日本語っていう暗号',
 '-6K2nN9aWsg-00017122-00017391': '特に英語との関係を考えるとですね',
 '-6K2nN9aWsg-00018528-00018718': 'またミルクに戻りますね',
 '-6K2nN9aWsg-00021816-00021960': 'そんなどうでもいい',
 '-6K2nN9aWsg-00024433-00024727': '例えばえあの小さい例なんですけどえーっと',
 '-6K2nN9aWsg-00026633-00026991': '軽蔑ガイダンスまきっといじめ問題があったに違いないと',
 '-6K2nN9aWsg-00028077-00028482': '少しずつ使えるように日本語覚えていきました',
 '-6K2nN9aWsg-00029963-00030450': 'でこういう風にこうあの身を異文化に置くというのは',
 '-6K2nN9aWsg-00032443-00032652': 'そこに踏み出すというのは',
 '-6K2nN9aWsg-00034689-00035192': 'インターネットを見るとこういうこういうのはいっぱいえー載ってると思うんですけど',
 '-6K2nN9aWsg-00038008-00038267': 'えー見ていきたいと思うんです',
 '-6K2nN9aWsg-00039710-00039905': 'まあ要するにねえ天才じゃない',
 '-6K2nN9aWsg-00040918-00041174': '共感できる部分あると思うんで

CER: 0.1429


In [13]:
# 10個サンプルする
import random
file_id_list = random.sample(id2text.keys(), 10)

paths2audio_files = [str(wav_dir / f'{file_id}.wav') for file_id in file_id_list]
asr_text = model.transcribe(paths2audio_files=paths2audio_files, batch_size=1, return_hypotheses=False)

[NeMo W 2024-05-29 14:28:52 nemo_logging:349] /tmp/ipykernel_31770/2328277468.py:3: DeprecationWarning: Sampling from a set deprecated
    since Python 3.9 and will be removed in a subsequent version.
      file_id_list = random.sample(id2text.keys(), 10)
    


Transcribing:   0%|          | 0/10 [00:00<?, ?it/s]

Beam search progress:: 100%|██████████| 1/1 [00:00<00:00, 32.70sample/s]


In [16]:
asr_text[0]

['そうするとふだんと違う環境を与えることによって白日の下にさらし出してしまうわけです。',
 '皆さんの考えるそれぞれの答えが。',
 'こういった火星に探査船を送るようなプロジェクトと',
 '本当にやるんですよ。',
 'ここからは私の考える存在というものについて',
 'プログラムをこちらで組んであげないと何もしてくれないんです。',
 '話し合いました。',
 '滝のように落ちていきます。',
 '装置を作ってみないと分からなくて。',
 'とだったので。']

In [15]:
gt_text = [id2text[file_id] for file_id in file_id_list]
gt_text

['普段と違う環境を与えることによってまあ白日のもとに晒し出してしまう訳です',
 '皆さんの考えるそれぞれの答えが',
 'というのはこういった火星に探査船を送るようなプロジェクトというのは',
 'いやこれほんとですよほんとにやるんですよ',
 'ここからは私の考える存在というものについて',
 'プログラムをこちらで組んであげないと何もしてくれないんです',
 'そういうことを話し合いました',
 'そのまま滝のようにえー落ちていきます',
 '例えばまあ装置を作ってみないと分からなかった',
 'とのことだったので']

In [18]:
calculate_cer(gt_text, asr_text[0])

0.31797235023041476